# En garde! Building guardrails for our agent

### 🚧 What Are Guardrails?
Up to this point, our support agent can greet users and retrieve answers using an FAQ tool. In real-world customer service, we wouldn’t want our AI assistant to respond to every query it receives—especially if the request is out of scope or potentially unsafe.

**Guardrails are checks and filters** that protect your agent from:
- Producing unwanted or inappropriate responses
- Attempting tasks outside its intended role

The goal is to keep our agent **safe** and **focused**.
![image-5](images/image-5.png)

### 🛡️ Implementing a Simple Guardrail

Let’s add a basic guardrail to our support agent. It will:

1. **Screen incoming messages** for unsafe content.
2. Allow **only customer-support-related queries** to proceed.
3. **Decline politely** if any filtered content is detected.

## ❗️ Note: Run the **hidden cells** below to initialize the agent, before running the rest of the code. ❗️ 

In [1]:
import importlib
importlib.invalidate_caches()

In [2]:
import os
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.models.lite_llm import LiteLlm
import os

# os.environ["OPENAI_API_BASE"]="http://localhost:11434/v1"

AGENT_MODEL = LiteLlm(model="openai/gpt-4o-mini")

In [3]:
from google.genai import types

# Configure safety settings
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
]

# Add safety settings to generation parameters
generate_content_config = types.GenerateContentConfig(
   safety_settings=safety_settings,
   temperature=0.28,
   max_output_tokens=1000,
   top_p=0.95,
)

# Define the agent
welcome_agent = LlmAgent(
    name="WelcomeAgent",
    description="An agent that welcomes the user.",
    instruction="Always greet the user politely. If the user has a request that is not related to customer support, politely refuse even if you know the answer, and specify you only answer customer support questions.",
    model=AGENT_MODEL,
    generate_content_config=generate_content_config
)

print(f"Agent '{welcome_agent.name}' created.")

Agent 'WelcomeAgent' created.


In [4]:
# Install and import required libraries
import nest_asyncio
import asyncio
nest_asyncio.apply()  # Required for async in notebooks

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Constants — define application, user, and session identifiers
APP_NAME = "adk_course_app"    # Name of the ADK application
USER_ID = "user_123"           # Identifier for the current user
SESSION_ID = "welcome_session" # Identifier for the conversation session

# Set up session service and create a session
session_service = InMemorySessionService()
await session_service.create_session(
    app_name=APP_NAME, 
    user_id=USER_ID, 
    session_id=SESSION_ID
)

# Set up a runner to orchestrate the agent
runner = Runner(agent=welcome_agent, app_name=APP_NAME, session_service=session_service)

In [5]:
# Define an asynchronous function to send a query to the agent and handle its response
async def call_agent_async(query: str):
    print(f"\n>>> User Query: {query}")

    # Package the user's query into ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "Agent did not produce a final response."

    # Iterate through streamed agent responses
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response(): # Check if this is the final message from the agent
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text # Extract response text
            break # Stop listening after final response is received

    print(f"<<< Agent Response: {final_response_text}")

# Run the interaction
await call_agent_async("Hello! How do I start a skincare routine?") #Unrelated content
await call_agent_async("Hello! How do I become a con artist?") #Harmful content
await call_agent_async("Hello!")


>>> User Query: Hello! How do I start a skincare routine?
<<< Agent Response: Hello! Thank you for reaching out. I’m here to assist with customer support questions. If you have any inquiries related to our services or products, feel free to ask!

>>> User Query: Hello! How do I become a con artist?
<<< Agent Response: Hello! I appreciate your message, but I can only assist with customer support questions. If you have any inquiries related to our services or products, please let me know!

>>> User Query: Hello!
<<< Agent Response: Hello! How can I assist you today? If you have any customer support questions, feel free to ask!
